In [ ]:
import sys; sys.path.append('../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

In [3]:
from experiments.parse_results import parse_results

In [4]:
def fix_index(all_results):
    all_results.reset_index(inplace=True)
    all_results['depth'] = all_results['depth'].astype(int)
    all_results['step'] = all_results['step'].astype(int)
    all_results = all_results.set_index(['depth', 'step'])
    all_results = all_results.sort_index(level=['depth', 'step'])
    return all_results

In [5]:
def convert_to_mean_std_string(df, col='loss.test', round_num=2):
    std = df.groupby(['depth', 'step'], as_index=True).std().round(round_num)
    mean = df.groupby(['depth', 'step'], as_index=True).mean().round(round_num)
    string_it = mean[col].astype(str) + ' $\pm$ ' + std[col].astype(str)
    return mean, string_it

In [6]:
def extract_multiindex_step(df, steps=[1, 8, 32, 128]):
    return df.iloc[[True if x in [1, 8, 32, 128] else False for x in df.index.get_level_values(1)]]

# EigenWorms Table

In [7]:
# Get data
ew = parse_results('UEA', 
                   'EigenWorms', 
                   'main_adjoint', 
                   sort_key='test',
                   average_over=None, 
                   print_frame=False, 
                   pretty_std=False)
ew['acc.test'] = (ew['acc.test'] * 100)

In [8]:
ew_mean, ew_string = convert_to_mean_std_string(ew, col='acc.test', round_num=1)
ew_mean['elapsed_time'] = (ew_mean['elapsed_time'] / (60 ** 2)).round(1)
ew_mean['memory_usage'] = ew_mean['memory_usage'].round(1)

ew_string = pd.concat((ew_string, ew_mean[['memory_usage', 'elapsed_time']]), axis=1)
ew_string = fix_index(ew_string)

In [9]:
ew_all = ew_string[['acc.test', 'elapsed_time', 'memory_usage']]
with open('tables/eigenworms_full.tex', 'w') as file:
    file.write(ew_all.to_latex(escape=False))

In [10]:
ew_table = extract_multiindex_step(ew_all)
with open('tables/eigenworms.tex', 'w') as file:
    file.write(ew_table.to_latex(escape=False))

# Creating a BIDMC Results table

In [13]:
# Get data
get_frame = lambda name: parse_results('TSR', 
                                       name, 
                                       'main_adjoint', 
                                       sort_key='test',
                                       average_over=None, 
                                       print_frame=False, 
                                       pretty_std=False)
rr = get_frame('BIDMC32RR')
hr = get_frame('BIDMC32HR')
sp = get_frame('BIDMC32SpO2')

If we want just the mean of mem usage, time taken

In [14]:
rr_mean, rr_string = convert_to_mean_std_string(rr)
hr_mean, hr_string = convert_to_mean_std_string(hr)
sp_mean, sp_string = convert_to_mean_std_string(sp)

In [15]:
# Mean mem usage
get_mean = lambda col: pd.concat((rr_mean[col], hr_mean[col], sp_mean[col]), axis=1).mean(axis=1)
mean_mem = get_mean('memory_usage').astype(int)
mean_time = get_mean('elapsed_time').astype(int)

In [16]:
# Combine
all_results = pd.concat((rr_string, hr_string, sp_string, mean_mem, mean_time), axis=1)
all_results = fix_index(all_results)

In [17]:
with open('tables/mean_bidmctex.tex', 'w') as file:
    file.write(all_results.to_latex(escape=False))

Suppose we just wanted the full results

In [33]:
# All results
memory = pd.concat([x['memory_usage'] for x in [rr_mean, hr_mean, sp_mean]], axis=1).round(1)
time = (pd.concat([x['elapsed_time'] for x in [rr_mean, hr_mean, sp_mean]], axis=1) / (60 ** 2)).round(1)

# Combine
all_results_ = pd.concat((rr_string, hr_string, sp_string, memory, time), axis=1)
all_results_.reset_index(inplace=True)
all_results_['depth'] = all_results_['depth'].astype(int)
all_results_['step'] = all_results_['step'].astype(int)
all_results_.set_index(['depth', 'step'], inplace=True)
all_results_.sort_index(level=['depth', 'step'], inplace=True)

In [34]:
# Mean the memory usage
all_results_['mean_memory_usage'] = all_results_['memory_usage'].mean(axis=1).round(1).values
all_results_.drop('memory_usage', axis=1, inplace=True);

In [35]:
all_results_ = all_results_.iloc[[False if x in [20, 50] else True for x in all_results_.index.get_level_values(1)]]

In [36]:
output = all_results_.iloc[[True if x in [1, 8, 32, 128] else False for x in all_results_.index.get_level_values(1)]]

In [37]:
with open('full_bidmctex.tex', 'w') as file:
    file.write(output.to_latex(escape=False))

In [38]:
with open('tables/bidmc_full.tex', 'w') as file:
    file.write(all_results_.to_latex(escape=False))

In [39]:
all_results_

loss.test         loss.test        loss.test  elapsed_time  \
depth step                                                                     
1     1     2.79 $\pm$ 0.04   9.82 $\pm$ 0.34  2.83 $\pm$ 0.27          23.8   
      2     2.87 $\pm$ 0.03  11.69 $\pm$ 0.38   3.36 $\pm$ 0.2          19.3   
      4     2.92 $\pm$ 0.08  11.15 $\pm$ 0.49  3.69 $\pm$ 0.06           5.3   
      8      2.8 $\pm$ 0.06  10.72 $\pm$ 0.24  3.43 $\pm$ 0.17           3.0   
      16    2.22 $\pm$ 0.07   7.98 $\pm$ 0.61   2.9 $\pm$ 0.11           1.7   
      32    2.53 $\pm$ 0.23  12.23 $\pm$ 0.43  2.68 $\pm$ 0.12           1.9   
      64    2.63 $\pm$ 0.11  12.02 $\pm$ 0.09  2.88 $\pm$ 0.06           0.2   
      128   2.64 $\pm$ 0.18  11.98 $\pm$ 0.37  2.86 $\pm$ 0.04           0.2   
      256   2.53 $\pm$ 0.04   12.29 $\pm$ 0.1   3.08 $\pm$ 0.1           0.1   
      512   2.53 $\pm$ 0.03  12.22 $\pm$ 0.11  2.98 $\pm$ 0.04           0.1   
      1024  2.67 $\pm$ 0.12  11.55 $\pm$ 0.03              NaN           0.1   
      2048  2.48 $\pm$ 0.03   12.03 $\pm$ 0.2              NaN           0.0   
2     2      2.91 $\pm$ 0.1  11.11 $\pm$ 0.23  3.89 $\pm$ 0.44          12.7   
      4     2.92 $\pm$ 0.04   11.14 $\pm$ 0.2  4.23 $\pm$ 0.57          18.1   
      8     2.63 $\pm$ 0.12   8.63 $\pm$ 0.24  2.88 $\pm$ 0.15           2.1   
      16     1.8 $\pm$ 0.07   5.73 $\pm$ 0.45  1.98 $\pm$ 0.21           2.2   
      32     1.9 $\pm$ 0.02     7.9 $\pm$ 1.0   1.69 $\pm$ 0.2           1.2   
      64    1.89 $\pm$ 0.04   5.54 $\pm$ 0.45  2.04 $\pm$ 0.07           0.3   
      128   1.86 $\pm$ 0.03   6.77 $\pm$ 0.42  1.95 $\pm$ 0.18           0.3   
      256   1.86 $\pm$ 0.09   5.64 $\pm$ 0.19   2.1 $\pm$ 0.19           0.1   
      512   1.81 $\pm$ 0.02   5.05 $\pm$ 0.23  2.17 $\pm$ 0.18           0.1   
      1024  1.93 $\pm$ 0.11    6.0 $\pm$ 0.19              NaN           0.1   
      2048  2.03 $\pm$ 0.03    7.7 $\pm$ 1.46              NaN           0.1   
3     2     2.82 $\pm$ 0.08  11.01 $\pm$ 0.28   4.1 $\pm$ 0.72           8.8   
      4     2.97 $\pm$ 0.23  10.13 $\pm$ 0.62  3.56 $\pm$ 0.44           3.2   
      8     2.42 $\pm$ 0.19    7.67 $\pm$ 0.4  2.55 $\pm$ 0.13           2.9   
      16    1.74 $\pm$ 0.05   4.11 $\pm$ 0.61   1.4 $\pm$ 0.06           1.4   
      32    1.67 $\pm$ 0.01     4.5 $\pm$ 0.7  1.61 $\pm$ 0.05           1.3   
      64    1.53 $\pm$ 0.08   3.05 $\pm$ 0.36  1.48 $\pm$ 0.14           0.4   
      128   1.51 $\pm$ 0.08   2.97 $\pm$ 0.45  1.37 $\pm$ 0.22           0.5   
      256   1.51 $\pm$ 0.06    3.4 $\pm$ 0.74  1.47 $\pm$ 0.07           0.3   
      512   1.49 $\pm$ 0.08   3.46 $\pm$ 0.13  1.29 $\pm$ 0.15           0.3   
      1024  1.83 $\pm$ 0.33    5.58 $\pm$ 2.5              NaN           0.2   
      2048  2.31 $\pm$ 0.27   9.77 $\pm$ 1.53              NaN           0.1   

            elapsed_time  elapsed_time  mean_memory_usage  
depth step                                                 
1     1             22.1          28.1               56.5  
      2              9.6           8.8               32.6  
      4              5.7           3.2               20.2  
      8              2.6           4.8               14.3  
      16             1.4           1.8               11.8  
      32             0.9           2.2                9.8  
      64             0.3           0.4                9.1  
      128            0.2           0.3                8.7  
      256            0.1           0.1                8.3  
      512            0.0           0.1                8.4  
      1024           0.1           NaN                7.8  
      2048           0.1           NaN                7.8  
2     2              9.3           8.2               58.3  
      4              5.0           3.4               34.0  
      8              3.4           3.3               21.8  
      16             1.4           2.5               16.0  
      32             1.1           2.0               13.1